In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import statsmodels.api as sm

In [5]:
def read_goog_sp_500_dataframe():
    googFile = 'data/GOOG.csv'
    spFile = 'data/GSPC.csv'
    
    goog = pd.read_csv(googFile, sep=",", usecols=[0,2], names=['Date', 'Goog'], header=0)
    sp = pd.read_csv(spFile, sep=",", usecols=[0,2], names=['Date', 'SP500'], header=0)
    goog['SP500'] = sp['SP500']
    
    # Format the date date object into a datetime
    goog['Date'] = pd.to_datetime(goog['Date'], format='%Y/%m/%d')
    
    returns = goog[
        [key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ['float64', 'int64']]].pct_change()
    
    return returns

def read_goog_sp500_logistic_data():
    returns = read_goog_sp_500_dataframe() 
    returns['Intercept'] = 1
    xData = np.array(returns['SP500', 'Intercept'][1:-1])
    yData = (returns['Goog'] > 0[1:-1])
    
    return (xData, yData)

In [9]:
xData, yData = read_goog_sp500_logistic_data()

logit = sm.logit.fit()
predictions = (result.predict(xData) > 0.5)

# count the number of times the actual up days match the predicted up days
num_accurate_predictions = (list(ydata == predictions)).count(True)
pctAccuracy = float(num_accurate_predictions) / float(len(predctions))

print("Accuracy ", pctAccuracy)

In [16]:
xData, yData = read_goog_sp500_logistic_data()
W = tf.Variable(tf.ones([1, 2]), name="W")
b = tf.Variable(tf.zeros([2]), name="b")
x = tf.placeholder(tf.float32, [None, 1], name="x")
y_ = tf.placeholder(tf.float32, [None, 2], name="y_")
y = tf.matmul(x, W)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

all_xs = np.expand_dims(xData[:0], axis=1)
alll_ys = np.array([([1,0] if yEl == True else [0,1]) for yEl in ydata])
dataset_size = len(all_x)

def trainWithMultiplepointsPerEpoch(steps, train_step, batch_size):
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i in range(steps):
            if dataset_size == batch_size:
                batch_start_idx = 0
            elif dataset_size < batch_size:
                raise ValueError("dataset_size: %d, must be greater than batch_size: %d" % (dataset_size, batch_size))
            else:
                batch_start_idx = (i * batch_size) % (dataset_size)
            
            batch_end_idx = batch_start_idx + batch_size
            
            batch_xs = xData[batch_start_idx : batch_end_idx]
            batch_ys = yData[batch_start_idx : batch_end_idx]
            
            feed = {x: batch_xs.reshape(-1, 1), y_:batch_ys.reshape(-1, 1)}
            
            sess.run(train_step, feed_dict=feed)
            
            if (i + 1) % 500 == 0:
                print("After %d iteration:" % i)
                print("W: %f" % sess.run(W))
                print("b: %f" % sess.run(b))
                
                print("cross entropy: %f" % sess.run(cross_entropy, feed_dict=feed))
                
        # test model 
        correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
        
        # calculate accuracy
        
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
trainWithMultiplePointsPointsPerEpoch(500, train_step, 100)

KeyError: ('SP500', 'Intercept')